# Functional Programming

## Functions Are Just Objects

What's the difference between an integer and a function?

In [ ]:
a = 42
def add_one(x):
    return x + 1

A function has a type, just like an integer:

In [ ]:
type(a), type(add_one)

A function has an identity, just like an integer:

In [ ]:
id(a), id(add_one)

`add_one` is just a _name_ of a function object, just as `a` is the _name_ of the integer object

In [ ]:
add_one, add_one(a)

A function has attributes, like other objects:

In [ ]:
dir(add_one)

Note the `__call__()` magic method:

In [ ]:
add_one.__call__(42)

In [ ]:
from dataclasses import dataclass

We can define classes having a `__call__()` method

In [ ]:
@dataclass
class AddN:
    n: int
    
    def __call__(self, x) -> int:
        return x + self.n

... and call objects of these classes as if they were functions:

In [ ]:
add_two = AddN(2)
add_two(42)

So a function is nothing else than an object with a `__call__()` method implemented. Every mention of _callable_ in documentation or type hints means that the "callable" object has a `__call__()` implementation.

In [ ]:
callable(add_one), callable(add_two)

__Discussion__: What would be a good use case for having an (obviously more complex) class with a `__call__()` method instead of a simple function definition?

## Functions as Arguments

Because functions are objects, we can pass them as arguments to other functions:

In [ ]:
from typing import Callable, Iterable, TypeVar, Any

T = TypeVar('T')

def mapper(fn: Callable[[T], T], l: Iterable[T]) -> list[T]:
    return [fn(x) for x in l]

Notice the `TypeVar` `T`, which allows us to specify that the exact types of elements of the iterables and callable signature are unknown, but they all must be of the same type. If we had used `Any` instead of `T`, there would be no way to check if elements of `l` match the type of `fn`'s arguments.

In [ ]:
mapper(add_one, [1, 10, 100])

In [ ]:
def is_even(n: int) -> bool:
    return n % 2 == 0

__Exercise:__ Create a function `filters(fn, l)` that accepts a callable and iterable, just as `mapper()` above, and returns the elements of the iterable for which `fn` returns `True`. You can use the function `is_even()` to test your solution.

In [ ]:
# Your solution:


In [ ]:
assert filters(is_even, [1, 2, 3, 4]) == [2, 4]

Our `mapper()` and `filters()` functions were just there for illustration purposes: Python has the built-in functions [`map()`](https://docs.python.org/3/library/functions.html#map) and [`filter()`](https://docs.python.org/3/library/functions.html#filter).

In [ ]:
import pandas as pd

In [ ]:
transaction_df = pd.DataFrame({
    'amount': [42., 100., 999.],
    'from': ['bob', 'alice', 'bob'],
    'to': ['alice', 'bob', 'alice']
})
transaction_df

Let's define a function that indicates if the amount of a transaction is larger than 100:

In [ ]:
def select_large_transactions(transaction_df):
    return transaction_df['amount'] > 100  # Where does transaction_df refer to?

Since `.loc[]` accepts a _callable_ as input, we can pass our function to it:

In [ ]:
transaction_df.loc[select_large_transactions]

Note that we didn't _call_ the function ourselves. It was `.loc[]` that called our function, and it was `loc[]` that passed whatever DataFrame it was bound to at that moment as the first argument to our function.

We can similarly pass functions to `assign()`:

In [ ]:
def get_commission(transaction_df):
    return transaction_df['amount'] * 0.05

In [ ]:
transaction_df.assign(commission=get_commission)

We can also _freeze_ some arguments of a function before passing it using [`functools.partial()`](https://docs.python.org/3/library/functools.html#functools.partial) from the standard library:

In [ ]:
from functools import partial

In [ ]:
def add_n(x: int, n: int = 1):
    return x + n

add_three = partial(add_n, n=3)
add_three(10)

In [ ]:
def get_commission(transaction_df, commission_percent=5):
    return transaction_df['amount'] * (commission_percent / 100)

In [ ]:
transaction_df.assign(commission=partial(get_commission, commission_percent=10))

## Functions Returning Functions

If functions can accept other functions as arguments, they surely can also _return_ functions:

In [ ]:
def add_n(n: int) -> Callable[[int], int]:
    def adder(x: int) -> int:
        return x + n
    return adder

In [ ]:
add_two = add_n(2)

In [ ]:
add_two, type(add_two)

In [ ]:
add_two(42)

In [ ]:
mapper(add_two, [1, 42, 100])

In [ ]:
mapper(add_two, [1, None, 100])

There's a problem when we pass `None` (or other unexpected types) to the function that we apply on our iterable. We can fix it by modifying the adder function:

In [ ]:
def add_one(x: int) -> int:
    if x is not None:
        return x + 1
    else:
        return None

mapper(add_one, [1, None, 100])

Why is this sub-optimal? What if there's an `add_two()` etc? Do we need to repeat all the checking for None everywhere?

In [ ]:
def skip_None(fn: Callable[[int | None], int | None]) -> Callable[[int | None], int | None]:
    def fn_wrapper(n: int | None) -> int | None:
        if n is not None:
            return fn(n)
        else:
            return None
    
    return fn_wrapper

A solution is to define a _wrapper_ function `fn_wrapper()` that intercepts calls to the wrapped function `fn`, and only calls the wrapped function (and returns its result) if the arguments passed to the wrapped function are valid. Note that this wrapper function is defined inside the body of another function `skip_None()`, which accepts the actual function to be wrapped.

In [ ]:
def add_one(x: int) -> int:
    return x + 1

def add_two(x: int) -> int:
    return x + 2

add_one_wrapped = skip_None(add_one)
add_two_wrapped = skip_None(add_two)
mapper(add_one_wrapped, [1, None, 100]), mapper(add_two_wrapped, [1, None, 100])

Instead of manually calling `skip_None()` to wrap our adder functions as above, Python has the `@` _decorator_ construct, which is just a syntactic shortcut to help us write more concise code. The following snippets are all equivalent:

In [ ]:
def add_one(x: int) -> int:
    return x + 1

skip_None(add_one)(1), skip_None(add_one)(None)

In [ ]:
def add_one(x: int) -> int:
    return x + 1

add_one = skip_None(add_one)
add_one(1), add_one(None)

In [ ]:
@skip_None
def add_one(x: int) -> int:
    return x + 1

add_one(1), add_one(None)

We can also write decorators that accept arguments. These add a second layer of inner function nesting:

In [ ]:
def check_value(
    max_expected: float
) -> Callable[[Callable[[float], float]], Callable[[float], float]]:  # 🤯
    def value_check_decorator(
        fn: Callable[[float], float]
    ) -> Callable[[float], float]:
        def fn_wrapper(n: float) -> float:
            if n > max_expected:
                print(f'Unusual value {n}, expected a maximum of {max_expected}')
            return fn(n)
        return fn_wrapper
    return value_check_decorator

@check_value(max_expected=42)
def add_one(x: int) -> int:
    return x + 1

mapper(add_one, [1, 42, 100])

Seeing decorators like these for the first time may be confusing. The following snippets clarify the nesting of functions:

In [ ]:
def add_one(x: int) -> int:
    return x + 1

max_41_checker = check_value(max_expected=41)
add_one_with_max_41_checking = max_41_checker(add_one)
add_one_with_max_41_checking(40), add_one_with_max_41_checking(42)

In [ ]:
(
    check_value(max_expected=41)(add_one)(40),
    check_value(max_expected=41)(add_one)(42)
)

## Anonymous Functions

When we intend to use a one-off function that consists of returning the evaluation of one expression, we can use _lambda expressions_ instead of defining a (named) function:

In [ ]:
mapper(lambda x: x + 10, [1, 42, 100])

Even a lambda expression ... is just an object (of type `function`)!

In [ ]:
type(lambda x: x + 10)

In [ ]:
id(lambda x: x + 10)

In [ ]:
dir(lambda x: x + 10)

In [ ]:
(lambda x: x + 10)(42)

We can even assign a lambda expression to a variable, but this is discouraged in the "official" [Python Style Guide](https://www.python.org/dev/peps/pep-0008/).

In [ ]:
# Don't do this IRL
add_ten = lambda x: x + 10
add_ten(42)

Lambda expressions can have more than one argument:

In [ ]:
from functools import reduce

In [ ]:
reduce(lambda x, y: x + y, [1, 2, 3])

## Exercises

Let's look back at our Vector class from the previous module.

In [ ]:
from dataclasses import dataclass
from __future__ import annotations

In [ ]:
@dataclass
class Vector:
    values: list[float]
    
    def __getitem__(self, index: int) -> float:
        return self.values[index]
    
    def __len__(self) -> int:
        return len(self.values)
    
    def __mul__(self, scalar: float) -> Vector:
        return Vector([v * scalar for v in self.values])
    
    def __add__(self, other: Vector) -> Vector:
        return Vector([self[i] + other[i] for i in range(len(self))])

__Exercise__: Add a method `pipe()` to our `Vector` class, which accepts as argument a function that transforms its input vector argument to another vector. As an example of such transformation functions, consider `rotate_right()` and `rotate_left()` given below. Test your implementation with the assertions below.

In [ ]:
def rotate_right(v: Vector) -> Vector:
    # rotates input by 90 degrees clockwise
    return Vector([v[1], -v[0]])

def rotate_left(v: Vector) -> Vector:
    # rotates input by 90 degrees counterclockwise
    return Vector([-v[1], v[0]])

In [ ]:
# Your solution:

In [ ]:
# %load solutions/vector_pipe.py

In [ ]:
assert Vector([1, 1]).pipe(rotate_right) == Vector([1, -1])
assert Vector([1, 1]).pipe(rotate_left).pipe(rotate_left) == Vector([-1, -1])

__Bonus Exercise__: Make sure that our `pipe()` method can accept additional arguments: create a generic `rotate()` function that accepts an optional `direction` keyword argument with possible values `'right'`, `'clockwise'`, `'left'`, `'counterclockwise'`. If an invalid or no `direction` keyword argument is provided, `rotate()` should return the original vector. See the assertions below for the expected behavior of the solution.

In [ ]:
# Your solution:

In [ ]:
# %load solutions/vector_pipe_vararg.py

In [ ]:
# bonus, have one generic rotation function
assert Vector([1, 1]).pipe(rotate) == Vector([1, 1])
assert Vector([1, 1]).pipe(rotate, direction='right').pipe(rotate, direction='counterclockwise') == Vector([1, 1])
assert Vector([1, 1]).pipe(rotate, direction='clockwise').pipe(rotate, direction='left') == Vector([1, 1])

In [ ]:
# bonus, deal with any Vector manipulation function
assert Vector([1, 1]).pipe(lambda v: Vector([v[0] * 42, v[1] * 99])) == Vector([42, 99])

By now, the pandas `pipe()` method should have no secrets anymore:

In [ ]:
transaction_df

In [ ]:
def select_amount_greater_than(tx_df, amount=100):
    return tx_df.loc[lambda df: df['amount'] > amount]

In [ ]:
transaction_df.pipe(select_amount_greater_than)

In [ ]:
transaction_df.pipe(select_amount_greater_than, amount=99)

In [ ]:
transaction_df.pipe(lambda df: df.loc[df['to'].isin(['bob', 'carol'])])